In [20]:
import theano

Using gpu device 0: GeForce GT 740M (CNMeM is disabled, CuDNN 5105)
/home/nvvaulin/.local/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your CuDNN version is more recent then Theano. If you see problems, try updating Theano or downgrading CuDNN to version 4.
  warnings.warn(warn)


In [21]:
import numpy as np
import pandas as pd
from StringIO import StringIO
import re
import os
import pickle
import random


class Cell:
    def __init__(self,v,p,fp,e):
        self.e = e
        self.p = p.copy()
        self.fp = fp.copy()
        self.v = v.copy()
        
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)
all_data = load_obj('all_data')
all_data = [i for i in all_data if 6 < len(i.p)<= 64]

In [22]:

import theano.tensor as T
import lasagne
from lasagne.layers import *
from broadcast import *
from lasagne.nonlinearities import *

import broadcast

In [23]:
def make_theta_R(v,p):#v:bs,v_num,axis
    theta = np.empty((len(v),2,p.shape[1],p.shape[1],p.shape[1]),dtype=np.float32)
    R = np.empty((len(v),1,p.shape[1],p.shape[1]),dtype=np.float32)
    v_norm = v/np.sqrt((v**2).sum(-1))[...,None]
    pos = (v_norm[:,None,:,:]*p[:,:,None,:]).sum(-1)
    R[:,0,:,:] = np.sqrt(((pos[:,:,None,:] - pos[:,None,:,:])**2).sum(-1))
    dist = pos[:,:,None,:]-pos[:,None,:,:]
    for i in range(dist.shape[1]):
        dist[:,i,i,:] = 1.
    dist /= np.sqrt((dist**2).sum(-1))[...,None]
    
    for i in range(dist.shape[1]):
        dist[:,i,i,:] = 0.
    theta[:,0,:,:,:] = (dist[:,:,None,:]*dist[:,None,:,:]).sum(-1)
    
    theta[:,1,:,:,:] = ((dist[:,:,:,None,1]*dist[:,:,None,:,2])-(dist[:,:,:,None,2]*dist[:,:,None,:,1]))**2+\
                       ((dist[:,:,:,None,0]*dist[:,:,None,:,2])-(dist[:,:,:,None,2]*dist[:,:,None,:,0]))**2+\
                       ((dist[:,:,:,None,0]*dist[:,:,None,:,1])-(dist[:,:,:,None,1]*dist[:,:,None,:,0]))**2
    return theta,R
    

def train_data_generator(train_data,_batch_size,batches_per_epoch):
    natoms = []
    unique,natoms,counts = np.unique(np.array([len(i.p) for i in train_data]),return_inverse=True,return_counts=True)
    unique_inx = np.arange(len(unique),dtype=np.int)
    natom_inx = []  
    tmp = np.arange(len(natoms),dtype=np.int)
    for i in unique_inx:
        natom_inx.append(tmp[natoms==i])
    w = counts.astype(np.float32)
    w /= w.sum()
    for batch_num in xrange(batches_per_epoch):
        count = np.random.choice(unique_inx,p=w)
        
        if(unique[count] <= 12):
            batch_size = _batch_size
        else:
            batch_size = 1
            
        inx = np.random.choice(natom_inx[count],batch_size)        
        batch_y = np.empty((batch_size,1),dtype=np.float32)
        batch_p = np.empty((batch_size,unique[count],3),dtype=np.float32)
        batch_v = np.empty((batch_size,3,3),dtype=np.float32)
        for j,i in enumerate(inx):
            batch_p[j] = train_data[i].p
            batch_v[j] = train_data[i].v
            batch_y[j,0] = train_data[i].e
        batch_y = (batch_y - energy_mean)/energy_var
        theta,R = make_theta_R(batch_v,batch_p)
        yield  theta,R,batch_y.reshape(-1,1)
        
def val_data_generator(data,_batch_size):
    lens = np.array([len(i.p) for i in data])
    inxs = np.argsort(lens)
    lens = lens[inxs]
    lo = 0
    while(lo < len(inxs)):
        natoms = lens[lo]
        if(natoms <= 12):
            batch_size = _batch_size
        else:
            batch_size = 1
        hi = min(lo+batch_size,np.searchsorted(lens[lo:],natoms,'right')+lo)    
        batch_y = np.empty((hi-lo,1),dtype=np.float32)
        batch_p = np.empty((hi-lo,natoms,3),dtype=np.float32)
        batch_v = np.empty((hi-lo,3,3),dtype=np.float32)
        for j,i in enumerate(inxs[lo:hi]):
            batch_p[j] = data[i].p
            batch_v[j] = data[i].v
            batch_y[j,0] = data[i].e
        lo = hi
        batch_y = (batch_y - energy_mean)/energy_var
        theta,R = make_theta_R(batch_v,batch_p)
        yield  theta,R,batch_y.reshape(-1,1)

In [24]:

def train_test_split(data,val_ratio,seed = 0):
    inx = np.arange(len(data),dtype=int)
    np.random.seed(seed)
    np.random.shuffle(inx)
    sl = int(val_ratio*len(data))
    train_X = []
    val_X = []
    for i in range(sl):
        val_X.append(data[inx[i]])
    for i in range(sl,len(data)):
        train_X.append(data[inx[i]])
    return train_X,val_X

In [25]:

data_features = all_data
energy = np.array([i.e for i in  data_features])
energy_mean = energy.mean()
energy_var = np.sqrt(((energy-energy_mean)**2).mean())
train_data,val_data = train_test_split(data_features,0.1)

In [26]:

train_generator = lambda : train_data_generator(train_data,5,200)
val_generator = lambda : val_data_generator(val_data,5)

for i in train_generator():
    print i[0].max(),i[1].max(),i[2].max()
    break

1.0 12.368 -2.67013


In [27]:

from lasagne.layers import Layer
import numpy as np

class AxisSumLayer(Layer):
    def __init__(self, incoming, axis, **kwargs):
        self.incoming_ndim = len(incoming.output_shape)
        self.axis = axis
        assert self.axis < self.incoming_ndim
        super(AxisSumLayer, self).__init__(incoming, **kwargs)

    def get_output_for(self, input, **kwargs):
        self.symbolic_input_shape = input.shape
        res = T.sum(input,self.axis)
        return res

    def get_output_shape_for(self, input_shape):
        output_shape = [input_shape[i] for i in range(len(input_shape)) if i != self.axis]
        return tuple(output_shape)

def shuffle(axis,_shape):
    return [_shape[axis]]+_shape[:axis]+_shape[axis+1:]


def inv_shuffle(axis,_shape):
    shape = list(_shape)    
    return shape[1:axis+1]+[shape[0]]+shape[axis+1:]

class AxisMulLayer(lasagne.layers.MergeLayer):
    def __init__(self, incoming1, incoming2,axis, **kwargs):
        self.axis = axis
        super(AxisMulLayer, self).__init__([incoming1, incoming2], **kwargs)
        
    def get_output_shape_for(self, input_shapes):
        # (rows of first input x columns of second input)
        return self.input_shapes[0]
    
    def get_output_for(self, inputs, **kwargs):
        tmp = inputs[0].dimshuffle(shuffle(self.axis,range(len(list(inputs[0].shape)))))
        tmp = tmp*inputs[1]
        tmp = tmp.dimshuffle(inv_shuffle(self.axis,range(len(list(inputs[0].shape)))))
        return tmp
    

class AxisAddLayer(lasagne.layers.MergeLayer):
    def __init__(self, incoming1, incoming2,axis, **kwargs):
        super(AxisMulLayer, self).__init__([incoming1, incoming2], **kwargs)
        
    def get_output_shape_for(self, input_shapes):
        # (rows of first input x columns of second input)
        return self.input_shapes[0]
    
    def get_output_for(self, inputs, **kwargs):
        tmp = inputs[0].dimshuffle(shuffle(self.axis,list(inputs[0].shape)))
        tmp = tmp+inputs[1]
        tmp = tmp.simshuffle(inv_shuffle(self.axis,list(inputs[0].shape)))
        return tmp

In [37]:
def build(theta,R,theta_shape,R_shape):#theta: bs,nf,n,n,n R:bs,nf,n,n
    def make_dense(l,n,nonl = rectify):
        res = BatchNormLayer(l)
        res = DenseLayer(res,n,nonlinearity=nonl)
        return res
    
    
    
    theta_l = InputLayer(theta_shape,input_var = theta)
    bcast = BroadcastLayer(theta_l,[0,2,3,4])
    th_dense = bcast
    th_tanh   = UnbroadcastLayer(make_dense(th_dense,8,tanh),bcast)
    th_rect   = UnbroadcastLayer(make_dense(th_dense,8,rectify),bcast)
    
    R_l = InputLayer(R_shape,R)
    bcast = BroadcastLayer(R_l,[0,2,3])
    R_dense = bcast
    R_rect_m = UnbroadcastLayer(make_dense(R_dense,8,rectify),bcast)   
    R_rect_a = UnbroadcastLayer(make_dense(R_dense,16,rectify),bcast)
    
    thR1 = AxisMulLayer(th_tanh,R_rect_m,2)
    thR2 = AxisMulLayer(th_tanh,R_rect_m,3)
    thR3 = AxisMulLayer(th_tanh,R_rect_m,4)
    thR_sum = ElemwiseSumLayer([thR1,thR2,thR3,th_rect])
    bcast = BroadcastLayer(thR_sum,[0,2,3,4])
    thR_dense = make_dense(bcast,16)
    thR_sum = UnbroadcastLayer(thR_dense,bcast)
    
    gl_pool = AxisSumLayer(thR_sum,4)
    R1 = ElemwiseSumLayer([R_rect_a,gl_pool])
    
    bcast = BroadcastLayer(R1,[0,2,3])
    R1 = make_dense(bcast,16)
    R1 = UnbroadcastLayer(R1,bcast)
    
    gl_pool = AxisSumLayer(R1,3)
    bcast = BroadcastLayer(gl_pool,[0,2])
    gl_pool = make_dense(bcast,64)
    gl_pool = UnbroadcastLayer(gl_pool,bcast)
    
    gl_pool = AxisSumLayer(gl_pool,2)
    res = make_dense(gl_pool,1,nonl=identity)
    return res

theta = T.tensor5('theta')
R = T.tensor4('R')
energies = T.matrix('energy')
net = build(theta,R,(None,2,None,None,None),(None,1,None,None))

AttributeError: 'module' object has no attribute 'tensor5'

In [ ]:
pred = get_output(net)
params = get_all_params(net, trainable=True)
loss = lasagne.objectives.squared_error(pred,energies).mean()
G_lr = theano.shared(np.array(0.001, dtype=theano.config.floatX))
updates = lasagne.updates.adam(loss,params,G_lr)
print ('start')
train_fn = theano.function([theta,R,energies],[loss], allow_input_downcast=True, updates=updates)
print ('train_fn compiled')
test_fn = theano.function([theta,R,energies],[loss], allow_input_downcast=True)
print ('test_fn compiled')

In [14]:
def save_weights(network, name ):
    np.savez(name+".npz", **{"param%d" % i: param for i, param in enumerate(get_all_param_values(network))})
             
def load_weights(network,name ):
    f = np.load(name+".npz")
    params = [f["param%d" % i] for i in range(len(f.files))]
    f.close()
    set_all_param_values(network,params)

def train(num_epoch,train_data_genertor,val_data_genertor,
          train_fn=train_fn,
          test_fn=test_fn,
          net=net,
          model_prefix='model',
          G_lr = G_lr,
          lr_sh = [100,150]):
    
    log = open('train.log','a')
    
    for epoch in range(num_epoch):
        train_num_batch = 0.
        train_loss = 0.
        for i,batch in enumerate(train_data_genertor()):
            train_loss += train_fn(batch[0],batch[1],batch[2])[0]
            train_num_batch += 1.
            if(i % 10 == 0):
                print '\r %i %f'%(i,train_loss/train_num_batch),
                log.write('%i %f\n'%(i,train_loss/train_num_batch))
        val_loss = 0.
        val_num_batch = 0.
        for i,batch in enumerate(val_data_genertor()):
            val_loss += test_fn(batch[0],batch[1],batch[2])[0]
            val_num_batch += 1.
        save_weights(net,model_prefix+'repoch%itr%.3fval%.3f'%(epoch,train_loss/train_num_batch,val_loss/val_num_batch))
        print ('\repoch %i train_loss=%f val_loss=%f'%(epoch,train_loss/train_num_batch,val_loss/val_num_batch))
        log.write('epoch %i train_loss=%f val_loss=%f\n'%(epoch,train_loss/train_num_batch,val_loss/val_num_batch))
        
        if(epoch in lr_sh):
            G_lr.set_value(G_lr.get_value()*np.float(0.01))
        
    log.close()

In [15]:
val_generator = lambda : val_data_generator(val_data,30)
G_lr.set_value(np.float(0.01))
train(250,train_generator,val_generator)